<div>
    <img src="http://www.ient.rwth-aachen.de/cms/uploads/images/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

In [ ]:
# Copyright 2019 Institut für Nachrichtentechnik, RWTH Aachen University

#Numpy,Sys, Matplotlib Imports, display widgets correctly 
import sys
sys.path.insert(0,'./Bilder')
sys.path.insert(0,'../ient_python')
%matplotlib widget
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.pyplot import figure, draw, pause, close
from matplotlib.widgets import RectangleSelector

#iPython Imports
from ipywidgets import widgets,interact
import IPython.display as ip
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
from IPython.display import Video

#RWTH imports
import rwth_nb.plots.mpl_decorations as rwth_plt
import rwth_nb.misc.feedback as rwth_feedback

#Scikit-Image, cv2 Imports
from skimage.filters import threshold_otsu
from skimage.morphology import label, square,binary_erosion, binary_dilation, disk
from skimage.measure import regionprops
from skimage.draw import rectangle_perimeter
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.transform import rotate, rescale
from scipy.ndimage import binary_fill_holes
from cv2 import warpAffine, getRotationMatrix2D, resize, INTER_LINEAR

# Suppress warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

plt.close('all')
plt.rcParams.update({'figure.max_open_warning': 0})

## Teil 1: Klassifikation mit Formmerkmalen
In diesem Teil des Versuchs sollen Sie die wesentlichen Schritte zur Klassifikation von Objekten in Bildern mittels Formmerkmalen kennenlernen und durchführen.
Dazu werden Sie ein geladenes Bild zunächst binarisieren und das Rauschen im Bild reduzieren. Anschließend müssen die einzelnen Objekte des Bildes voneinander separiert und Merkmale berechnet werden.

Für nähere Informationen bezüglich der verwendeten Funktionen, schauen Sie hier:

__[Befehlsreferenz](Befehlsreferenz_V6.ipynb)__

Eine Übersicht des Praktikums und eine Einfürhung in diesen Versuch finden Sie in nachfolgendem Video.

In [ ]:
Video("http://www.ient.rwth-aachen.de/services/PTI-Videos/V6_T1_v4.mp4", width=480, height=270 )

<div class="alert rwth-topic">
    
## Aufgabe 1: Binarisierung
Ziel des Binarisierungsschrittes ist die Segmentierung eines Bildes in Vorder- und Hintergrundregionen.
    
</div>

<div class="alert rwth-subtopic">
    
### Aufgabe 1.1: Laden des Bildes 
Laden Sie zunächst das Bild unter dem Pfad 'Bilder/Buchstaben.png'. Speicheren Sie das Bild in einer Variable mit der Bezeichung *img*.
    
Wenn Sie dies erfolgreich gemacht haben, können Sie das Bild durch Ausführen der darauffolgenden Zelle darstellen.
  
</div>

In [ ]:
# Hier kommt Ihre Lösung zu Aufgabe 1.1:


In [ ]:
#Anzeigen des geladenen Bildes:
fig, ax = plt.subplots(1,1)
ax.imshow(img, cmap='gray');
ax.set_axis_off();
ax.set_title('Buchstaben');

<div class="alert rwth-subtopic">   
    
### Aufgabe 1.2: Histogramm
Berechnen Sie das Histogramm der Grauwerte im Bild. Verwenden Sie dabei 32 Bins. Schauen Sie sich das Histogramm an und geben Sie eine Schätzung für eine geeignete Binarisierungsschwelle an. Beachten Sie dabei, dass die Bildmatrix dabei als 1D Array vorliegen muss.
</div>

In [ ]:
# Diese zwei Zeilen legen Ihnen einen Plot "fig2" und eine Axis "ax" an. Dies sorgt für eine schöne Formatierung.
fig2, ax = plt.subplots()
rwth_plt.axis(ax);

#Hier kommt Ihre Lösung zu Aufgabe 1.2:


<div class="alert rwth-subtopic">
    
    
### Aufgabe 1.3: Ermittlung der Binarisierungsschwelle
Führen Sie die nächste Zelle aus. Beachten Sie, dass dafür eine Graustufenversion des Bildes unter dem Namen *img_gray* vorliegen muss. Finden Sie anschließend eine gute Binarisierungsschwelle durch Verschieben des Reglers.
        

</div>

In [ ]:
fig1 = figure(figsize=(10, 6));
ax1 = fig1.gca();

@widgets.interact(t=widgets.IntSlider(min=50, max=240, continuous_update=True))
def update_plot(t):
    if ax1.lines: # check if lines exist
        ax1.set_data(img_gray>t);
        ax1.draw()
    else:
        ax1.imshow(img_gray>t, cmap= 'gray')
        ax1.set_title("Finden der Binarisierungsschwelle")
        ax1.axis('off')

<div class="alert rwth-subtopic">
    
### Aufgabe 1.4: Methode von Otsu
Ermitteln Sie die optimale Binarisierungsschwelle nach der Methode von Otsu. Binarisieren Sie das Bild anschließend mit Hilfe der berechneten Schwelle und speichern Sie das entstandene Bild in einer Variable *img_bin*.
    
Wenn Sie dies erfolgreich gemacht haben, können Sie das Bild durch Ausführen der darauffolgenden Zelle darstellen.
        
</div>

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 1.4:


In [ ]:
#Plotten des binarisierten Bildes aus 1.4:
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(img_bin, cmap='gray');
ax.set_axis_off();
ax.set_title('Binarisiertes Bild nach Otsu');

<div class="alert rwth-topic">
    
## Aufgabe 2: Rauschreduktion mittels morphologischer Operationen
Ziel dieses Schrittes ist es, dass im Binärbild vorhandene Rauschen so weit wie möglich zu reduzieren. Dies soll mit Hilfe der morphologischen Bildverarbeitung erreicht werden.  
    
Im folgenden Video werden die morphologischen Operationen *Erosion* und *Dilatation* näher erläutert.

</div>

In [ ]:
Video("http://www.ient.rwth-aachen.de/services/PTI-Videos/Erosion_und_Dilatation.mp4", width=480, height=270 )

<div class="alert rwth-subtopic">
      
         
### Aufgabe 2.1: Binäre Erosion
Entfernen Sie störende Bildpunkte mit Hilfe einer binären Erosion. Experimentieren Sie dabei mit quadratischen Strukturelementen verschiedener Seitenlängen (3, 5, 10), sowie mit einem Kreis mit Radius 5. Wenn Sie die erodierten Bilder als *img_erode1* bis *img_erode4* bennenen, können Sie diese durch Ausführen der darauffolgende Zelle darstellen.
    
 * Welches Strukturelement bietet das beste Ergebnis? 
 * Welche Probleme können duch zu große oder zu kleine Strukturelemente auftreten? 
    
Verwenden Sie in den darauf folgenden Aufgaben das erodierte Bild mit dem besten Ergebnis. 
           

</div>

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 2.1:


In [ ]:
#Plotten der binären Erosionen aus 2.1:
fig, ax = plt.subplots(2,2,sharex='all', sharey='all', figsize=(14,14))
ax[0,0].imshow(img_erode1,cmap='gray')
ax[1,0].imshow(img_erode2,cmap='gray')
ax[0,1].imshow(img_erode3,cmap='gray')
ax[1,1].imshow(img_erode4,cmap='gray')
for axs in ax.flat:
    axs.set_axis_off()

<div class="alert rwth-subtopic"> 
    
### Aufgabe 2.2: Rekonstruktion
Die Erosion im vorhergehenden Schritt hat neben dem Rauschen auch zum Objekt gehörende Bildpunkte entfernt. Rekonstruieren Sie die ursprüngliche Form des Buchstabens durch eine Dilatation. Wenn Sie die resultierenden Bilder als *img_dil1* bis *img_dil4* bennenen, können Sie diese wieder durch Ausführen der darauffolgende Zelle darstellen.
    
 * Welches Strukturelement ist hier optimal?
 * Warum? 
    
Verwenden Sie in den nachfolgen Aufgaben das Bild mit dem besten Ergebnis.      
        
</div>

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 2.2:


In [ ]:
#Plotten der binären Dilatationen aus 2.2:

fig, ax = plt.subplots(2,2,sharex='all', sharey='all', figsize=(14,14))
ax[0,0].imshow(img_dil1,cmap='gray')
ax[1,0].imshow(img_dil2,cmap='gray')
ax[0,1].imshow(img_dil3,cmap='gray')
ax[1,1].imshow(img_dil4,cmap='gray')
for axs in ax.flat:
    axs.set_axis_off()

<div class="alert rwth-topic">
   
## Aufgabe 3: Labeling und Ausschneiden
Um die einzelnen Objekte im Bild mithilfe von Form-Merkmalen beschreiben zu können, müssen diese zunächst voneinander getrennt, also zumindest grob lokalisiert, werden.
Für das Beispiel hier reicht ein sehr einfaches Verfahren, bestehend aus Labeling und anschließendem Ausschneiden der Bounding-Box jedes gefundenen Objektes, aus.
       
</div>

<div class="alert rwth-subtopic">
        
### Aufgabe 3.1: Labeling
Das Labeling soll verschiedene Objekte im Bild voneinander trennen, indem es zusammenhängende Vordergrundregionen identifiziert. Ein Beispiel für ein Labelbild, wie es aus diesem Schritt resultieren soll, können Sie sich hier ansehen: __[Labelmatrix](./labelmatrix.ipynb)__ 
    
Auch hier können Sie das Resultat Ihrer Lösung durch Ausführen der darauffolgende Zelle darstellen, wenn Sie dies als *img_label* benennen.
                                                                                                               
</div>  

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 3.1:


In [ ]:
#Plotten des Label-Bildes aus 3.1:
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(img_label, cmap='gray');
ax.set_axis_off();
ax.set_title('labeled image');

<div class="alert rwth-subtopic">
     
### Aufgabe 3.2: Bounding-Box Bestimmen
Nach dem Labeling sollen die Objekte nun voneinander getrennt werden. Dafür soll hier zunächst eine Bounding-Box für jedes Objekt ermittelt werden.
    
 * Wie können die Bounding-Boxen dabei helfen, Merkmale veschiebungs- und skalierungsinvariant zu machen?
    
Speichern Sie die Koordinaten der Bounding-Box des ersten Buchstabens 'A' in den Variablen *minr*, *maxr*, *minc* und *maxc*.
    
</div>

In [ ]:
# Hier kommt ihre Lösung zu Aufgabe 3.2:


In [ ]:
#Plotten der Bounding Boxen aus 3.2:
fig, ax = plt.subplots(figsize=(10,10))
ax.axis('off')
ax.imshow(img_label, cmap= 'gray')
ax.set_title('Labeled Image with BoundingBox')
rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='yellow', linewidth=2)
ax.add_patch(rect)
plt.show()

<div class="alert rwth-subtopic">
    
### Aufgabe 3.3: Ausschneiden
Ab diesem Unterpunkt soll nur noch mit dem Buchstaben 'A' weitergearbeitet werden. Schneiden Sie dazu die Bounding-Box, die diesen enthält aus und speichern ihn in einer Variable namens *img_cropped*. Beachten Sie, dass an den Randstellen jeweils um ein Pixel erweitert werden muss, um den gesamten Buchstaben zu erhalten.
    
</div>

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 3.3:


In [ ]:
#Plotten  des ausgeschnittenen Buchstabens aus 3.3:
fig, ax = plt.subplots(1,1)
ax.imshow(img_cropped, cmap='gray');
ax.set_axis_off();
ax.set_title('cropped image');

<div class="alert rwth-topic">
    
## Aufgabe 4: Merkmalsbestimmung am Beispiel Homogenität 
Ziel dieser Aufgabe ist es, dass Sie das Merkmal der Homogenität für den Buchstaben 'A' berechnen und sich mit dessen Eigenschaften beschäftigen.
        
Wie Objekte in Bildern mittels der Homogenität klassifiziert werden können, ist in nachfolgendem Video erläutert.
</div>

In [ ]:
Video("http://www.ient.rwth-aachen.de/services/PTI-Videos/Homogenitaet.mp4", width=480, height=270 )

<div class="alert rwth-subtopic">
    
### Aufgabe 4.1: Homogenität berechnen
Bestimmen Sie die Homogenität des Buchstabens 'A'. Berechnen Sie dafür sowohl die Fläche, als auch die Kontur. Beachten Sie dabei, dass die Homogenität sich immer auf die äußeren Konturen eines Objektes bezieht. 
       
Wenn Sie Zwischenergebnisse, die in Form von Bildern vorliegen, in Variablen namens *img_filled* und *img_contour* speichern, werden diese durch Ausführen der nächsten Zelle angezeigt.
</div>

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 4.1:


In [ ]:
#Plotten der Kontur und des gefüllten Buchstabens aus 4.1:
fig, ax = plt.subplots(1,2,sharex='all', sharey='all', figsize=(10,10))
ax[0].imshow(img_contour,cmap='gray')
ax[0].set_title('Kontur');
ax[1].set_title('Filled');
ax[1].imshow(img_filled,cmap='gray')
for axs in ax.flat:
    axs.set_axis_off()
fig.suptitle('Homogenität');

<div class="alert rwth-subtopic">
    
### Aufgabe 4.2: Homogenität - Eigenschaften
Führend Sie die nachfolgende Zelle aus und rotieren, sowie skalieren Sie die angezeigten Bilder mittels der Regler.
 *  Was stellen Sie fest?
 *  Ist die Homogenität invariant gegenüber Rotation und Skalierung? 
 *  Inwiefern sind diese Eigenschaften erstrebenswert bei der Merkmalsbeschreibung?
               
</div>

In [ ]:
kernelX = square(3)

def ient_homogenity(contour,image):
    homogenity = (np.sum(contour)*np.sum(contour))/np.sum(image)
    print('Homogenität: ',homogenity)
    

# load the cropped image embedded in bigger black img to avoid bound-crops and overlapping by neighboured characters
a_in_black = (imread("Bilder/A_in_black.png")).astype(bool)

# Rotate image correctly
def rotate_image(mat, angle):
    height, width = mat.shape[:2] 
    image_center = (width/2, height/2)

    rotation_mat = getRotationMatrix2D(image_center, angle, 1.)

    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])

    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    rotated_mat = warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat


fig, ax = plt.subplots(1,2,figsize=(10, 6))

# Rotate Slider
@widgets.interact(d=widgets.IntSlider(min=0, max=360, continuous_update=True))
def update_rotation(d):
        rotated_a = label(rotate_image(a_in_black.astype(np.double), d))
        regions = regionprops(rotated_a)
        boxes = np.array([label['BoundingBox']
                  for label in regions])
        rotated_a_cropped = rotated_a[boxes[0][0]-1:boxes[0][2]+1, boxes[0][1]-1:boxes[0][3]+1]
    
        if ax[0].lines: # check if lines exist
            ax[0].set_data(rotated_a_cropped);
            ax[0].draw()

        else:
            ax[0].imshow(rotated_a_cropped, cmap= 'gray')
            ax[0].set_title("Rotation des Buchstaben")
            ax[0].axis('off')
            
        print('Winkel: ',d, '°')
        rotated_filled = binary_fill_holes(rotated_a_cropped)
        rotated_contour = rotated_filled.astype(np.double) - binary_erosion(rotated_filled, kernelX)
        ient_homogenity(rotated_contour,rotated_filled)

#Resize Slider
@widgets.interact(r=widgets.FloatSlider(min=0.01, max=5, step=0.05,continuous_update=False))
def update_resize(r):
        # parameter for resizing
        width = int(a_in_black.shape[1] * r)
        height = int(a_in_black.shape[0] * r)
        dsize = (width, height)
        # resizing
        resized_image= resize(a_in_black.astype(float),dsize,interpolation = INTER_LINEAR)#INTER_NEAREST
        # crop to get A with little margin and cast to boolean
        resized_image_a = label(resized_image.astype(float))
        regions = regionprops(resized_image_a)
        boxes = np.array([label['BoundingBox']
                for label in regions])
        resized_image_cropped = resized_image_a[boxes[0][0]-1:boxes[0][2]+1, boxes[0][1]-1:boxes[0][3]+1].astype(bool)
        # plotting
        if ax[1].lines: # check if lines exist, if so: just update data
            ax[1].set_data(resized_image_cropped);
            ax[1].draw()
        else: # plot whole figure
            ax[1].imshow((resized_image_cropped), cmap= 'gray')
            ax[1].set_title("Skalierung des Buchstaben")
            ax[1].axis('off')
        print('Skalierungsfaktor: ',r)
        # homogenity
        scaled_filled = binary_fill_holes(resized_image)
        scaled_contour = scaled_filled.astype(np.double) - binary_erosion(scaled_filled, kernelX)
        ient_homogenity(scaled_contour,scaled_filled)
        

<div class="alert rwth-feedback">

    
# Feedback:

Liebe TeilnehmerInnen,

Wir würden uns freuen, wenn ihr am Ende jeder Aufgabe kurz eure Meinung aufschreibt. Ihr könnt auf die dadrunter liegende Zelle zu greifen und eure Anmerkungen zu der Aufgabe (oder auch generelles) reinschreiben.


</div>

In [ ]:
rwth_feedback.rwth_feedback('Feedback V6.1', [
    {'id': 'likes', 'type': 'free-text', 'label': 'Das war gut:'}, 
    {'id': 'dislikes', 'type': 'free-text', 'label': 'Das könnte verbessert werden:'}, 
    {'id': 'misc', 'type': 'free-text', 'label': 'Was ich sonst noch sagen möchte:'}, 
    {'id': 'learning', 'type': 'scale', 'label' : 'Ich habe das Gefühl etwas gelernt zu haben.'},
    {'id': 'supervision', 'type': 'scale', 'label' : 'Die Betreuung des Versuchs war gut.'},
    {'id': 'script', 'type': 'scale', 'label' : 'Die Versuchsunterlagen sind verständlich.'},
], "feedback.json", 'pti@ient.rwth-aachen.de')

# >>Weiter zu  [__Teil 2__](Teil_2.ipynb)
